# Aula 23/05

In [1]:
# importando bibliotecas necessárias
import pandas as pd
import numpy as np

# graphs
import matplotlib.pyplot as plt
import seaborn as sns

#imports
from sklearn.svm import SVC

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [2]:
def print_metrics(model_name: str, y_test, y_pred, y_test_pca, y_pred_pca):
  return (f'''
Acurácia do Modelo {model_name} sem PCA: {accuracy_score(y_test, y_pred):.4f}
Acurácia do Modelo {model_name} com PCA: {accuracy_score(y_test_pca, y_pred_pca):.4f}

Precisão do Modelo {model_name} sem PCA: {precision_score(y_test, y_pred, average='macro'):.4f}
Precisão do Modelo {model_name} com PCA: {precision_score(y_test_pca, y_pred_pca, average='macro'):.4f}

Recall (Sensibilidade) do Modelo {model_name} sem PCA: {recall_score(y_test, y_pred, average='macro'):.4f}
Recall (Sensibilidade) do Modelo {model_name} com PCA: {recall_score(y_test_pca, y_pred_pca, average='macro'):.4f}
''')

In [3]:
dataset = pd.read_csv('star_classification.csv')
dataset.head()

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   obj_ID       100000 non-null  float64
 1   alpha        100000 non-null  float64
 2   delta        100000 non-null  float64
 3   u            100000 non-null  float64
 4   g            100000 non-null  float64
 5   r            100000 non-null  float64
 6   i            100000 non-null  float64
 7   z            100000 non-null  float64
 8   run_ID       100000 non-null  int64  
 9   rerun_ID     100000 non-null  int64  
 10  cam_col      100000 non-null  int64  
 11  field_ID     100000 non-null  int64  
 12  spec_obj_ID  100000 non-null  float64
 13  class        100000 non-null  object 
 14  redshift     100000 non-null  float64
 15  plate        100000 non-null  int64  
 16  MJD          100000 non-null  int64  
 17  fiber_ID     100000 non-null  int64  
dtypes: float64(10), int64(7),

## Dataset Dict

obj_ID = Object Identifier, the unique value that identifies the object in the image catalog used by the CAS

alpha = Right Ascension angle (at J2000 epoch)

delta = Declination angle (at J2000 epoch)

u = Ultraviolet filter in the photometric system

g = Green filter in the photometric system

r = Red filter in the photometric system

i = Near Infrared filter in the photometric system

z = Infrared filter in the photometric system

run_ID = Run Number used to identify the specific scan

rereun_ID = Rerun Number to specify how the image was processed

cam_col = Camera column to identify the scanline within the run

field_ID = Field number to identify each field

spec_obj_ID = Unique ID used for optical spectroscopic objects (this means that 2 different observations with the same spec_obj_ID must share the output class)

class = object class (galaxy, star or quasar object)

redshift = redshift value based on the increase in wavelength

plate = plate ID, identifies each plate in SDSS

MJD = Modified Julian Date, used to indicate when a given piece of SDSS data was taken

fiber_ID = fiber ID that identifies the fiber that pointed the light at the focal plane in each observation


In [5]:
dataset.columns

Index(['obj_ID', 'alpha', 'delta', 'u', 'g', 'r', 'i', 'z', 'run_ID',
       'rerun_ID', 'cam_col', 'field_ID', 'spec_obj_ID', 'class', 'redshift',
       'plate', 'MJD', 'fiber_ID'],
      dtype='object')

In [6]:
X = dataset.drop(['obj_ID', 'run_ID', 'rerun_ID', 'field_ID', 'spec_obj_ID', 'plate', 'MJD', 'fiber_ID', 'cam_col', 'class'], axis=1)
y = dataset['class']

## convertendo class para inteiro
y = y.replace({'GALAXY': 0, 'QSO': 1, 'STAR': 2})

In [7]:
# normalizando os dados
X_std = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.3, random_state=42)


In [8]:
# pca
pca = PCA(n_components=7)
X_pca = pca.fit_transform(X)

X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.3, random_state=42)

### Modelo SVC

In [9]:
modelSVC = SVC(kernel= 'linear', random_state=42)
modelSVC.fit(X_train, y_train)

SVC(kernel='linear', random_state=42)

In [10]:
y_pred = modelSVC.predict(X_test)

In [ ]:
print(print_metrics('SVC', y_test, y_pred, y_test_pca, y_pred_pca))